In [ ]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal',
          {'theme': 'serif',
           'transition': 'slide',
           'start_slideshow_at': 'selected',
           'width': 1024,
           'height': 768});

# A tour of the ~~Scientific~~ ~~Oceanographic~~ Geosciences Python stack

![](./images/pork-knee.jpg)

### Before we begin

- Who am I?
- Who are you?
- What do you expect from this tutorial?

How to run this notebook


```bash
url=http://bit.ly/miniconda

wget $url -O miniconda.sh
bash miniconda.sh -b
export PATH=$HOME/miniconda/bin:$PATH
conda update --yes --all
```

```bash
url=http://bit.ly/conda-env

wget $url -O environment.yml
conda env create environment.yml
```

# NumPy

http://www.numpy.org/

See also:

- http://www.labri.fr/perso/nrougier/teaching/numpy.100/

Define an array

In [ ]:
import numpy as np
a = np.array([[1, 2, 3], [4, 5, 6]], dtype=np.complex)

print('Data type            : {}'.format(a.dtype))
print('Element size         : {}'.format(a.size))
print('Number of dimensions : {}'.format(a.ndim))
print('Shape                : {}'.format(a.shape))
print('Memory in bytes      : {}'.format(a.nbytes))

In [ ]:
np.arange(0, 2*np.pi, np.pi)

In [ ]:
np.linspace(0, 2*np.pi, 5)

In [ ]:
np.zeros_like(a) == np.zeros(a.shape)

Reading some data

In [ ]:
!head -n 5 ./data/dados_pirata.csv

We need to skip the header and the date-time columns

In [ ]:
data = np.loadtxt("./data/dados_pirata.csv",
                  skiprows=1,
                  usecols=range(2, 16),
                  delimiter=',')

data[data == -99999.] = np.NaN

data.shape, data.dtype

But the depth information was in header!

We need to type that in...

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

z = [1, 10, 100, 120, 13, 140, 180, 20,
     300, 40,5, 500, 60, 80]

fig, ax = plt.subplots()

ax.plot(data[42, :], z, 'ko') # Plot . because z was not ordered.
ax.invert_yaxis()

```python
import *
```

vs

```python
import numpy as np
```

In [ ]:
sum(range(5), -1)

In [ ]:
from numpy import *
sum(range(5), -1)

## Exercise

Load `dados_pirata.csv` again, but now use numpy masked arrays to remove the bad values.

In [ ]:
import numpy as np
import numpy.ma as ma

data = np.loadtxt("./data/dados_pirata.csv",
                  skiprows=1,
                  usecols=range(2, 16),
                  delimiter=',')

data[data == -99999.] = np.NaN

ma.masked_invalid(data)

## How to remember the methods, syntaxes, etc?

![](./images/tar.png)

http://mathesaurus.sourceforge.net/matlab-numpy.html

# Pandas: can we do better than NumPy arrays?

http://pandas.pydata.org/

Note that almost everything we did before, and some more, is now done at the loading time

In [ ]:
import pandas as pd

df = pd.read_csv('./data/dados_pirata.csv',
                 index_col='datahora',
                 parse_dates=True,
                 na_values=-99999)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.columns = ['{0:0>3}'.format(col.split('_')[1]) for
              col in df.columns]
df.sort_index(axis=1, inplace=True)

In [ ]:
df.head(5)

Pandas behave like masked-arrays,
but with some extra convenience functions beyond `nan{min,max,mean,etc}`

In [ ]:
df.dropna(how='all', axis=1, inplace=True)

df.head(3)

Quick way to explore the data

In [ ]:
desc = df.describe()
desc

![](./images/read_the_docs_until_the_end.jpg)

```
RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)
```

High level code!  No need for comments to describe what is being computed below:

In [ ]:
desc.ix['std'] ** 2

What levels are we plotting?

In [ ]:
ax = df[['001', '500']].plot(figsize=(11, 3))

More convenience functions:
- interpolate

In [ ]:
df['001'].interpolate().plot(figsize=(11, 3))
ax = df['001'].plot()

Full control of the interpolation

In [ ]:
kw = dict(method='time', limit=20)
df['001'].interpolate(**kw).plot(figsize=(11, 3))
ax = df['001'].plot()

Pandas loves time-series

In [ ]:
key = lambda x: x.month

grouped = df.groupby(key)

monthly = grouped.mean()

fig, ax = plt.subplots(figsize=(9, 5))

ax = monthly.plot(ax=ax)

## Exercise

Load the file `15t30717.3f1` and filter the tidal signal.

In [ ]:
from datetime import datetime
from pandas import read_table

cols = ['j', 'u', 'v', 'temp', 'sal', 'y', 'mn', 'd', 'h', 'mi']

df = read_table('./data/15t30717.3f1' , delim_whitespace=True, names=cols)
dates = [datetime(*x) for x in
         zip(df['y'], df['mn'], df['d'], df['h'], df['mi'])]
df.index = dates
df.drop(['y', 'mn', 'd', 'h', 'mi', 'j'], axis=1, inplace=True)
df.head()

In [ ]:
from pandas import rolling_mean

df = df.resample(rule='1H').mean()

df['low'] = df['v'].rolling(window=40, center=True).mean()
df['high'] = df['v'] - df['low']
df[['v', 'high', 'low']].plot()

## Pandas has a few siblings:

- geopandas
- ctd
- xray

GeoPandas read GeoJSONs, Shapefiles, etc as pandas DataFrames

http://geopandas.org/

In [ ]:
import geopandas as gpd

fname = "./data/2013-04-29-Running.geojson"

df = gpd.read_file(fname)
ax = df.plot()

The geometries are essentially shapely objects

In [ ]:
shape = df['geometry'][0]

msg = """
Length is {} km.

The center is locate at {!r}.

The track bounds are {}.
""".format
print(msg(shape.length * 111, shape.centroid.xy,
          shape.bounds))

[ptyhon-ctd](https://pypi.python.org/pypi/ctd) loads `ctd`, `xbt`, and many more ocean profilers directly into pandas

In [ ]:
from ctd import DataFrame
cast = DataFrame.from_cnv('./data/CTD_001.cnv.gz',
                          compression='gzip')
cast.head()

The augmented DataFrame has some extra methods to process CTD data

In [ ]:
downcast, upcast = cast['t090C'].split()
fig, ax = downcast.plot(figsize=(3, 7))

[xray](http://xray.readthedocs.org/en/stable/): `netCDF4` + `pandas` for >2D tables

In [ ]:
import xray

url = ('http://geoport.whoi.edu/thredds/dodsC/clay/usgs/'
       'users/jcwarner/Projects/Sandy/triple_nest/'
       '00_dir_NYB05.ncml') # './data/00_dir_NYB05.nc'

ds = xray.open_dataset(url)
ds

Slice the variables using the `var name` (like `netCDF4`)

In [ ]:
temp = ds['temp']
temp

Use high-level dates slice (like pandas)

In [ ]:
temp.sel(ocean_time='2012-10-25')

You can even slice using integers and named the axis

In [ ]:
t = temp.isel(s_rho=-1, ocean_time=0)

cs = t.plot()

# seaborn and statsmodels: Easy stats at you finger tips

https://stanford.edu/~mwaskom/software/seaborn/

http://statsmodels.sourceforge.net/

In [ ]:
import seaborn

from pandas import read_csv

kw = dict(na_values='NaN', sep=',', encoding='utf-8',
          skipinitialspace=True, index_col=False)

df = read_csv("./data/fish.csv", **kw)
df.head()

![](./images/educated_guess.jpg)

In [ ]:
kw = {'axes.edgecolor': '0', 'text.color': '0', 'ytick.color': '0', 'xtick.color': '0',
      'ytick.major.size': 5, 'xtick.major.size': 5, 'axes.labelcolor': '0'}

seaborn.set_style("whitegrid", kw)

corr = df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

ax = seaborn.heatmap(corr, mask=mask, square=True)

In [ ]:
import statsmodels.api as sm

results = sm.OLS(df['BDE 47 (ng/g)'], df['Days']).fit()

print(results.summary())

In [ ]:
ax = seaborn.jointplot("Days", "BDE 47 (ng/g)", df, kind="reg")

# dask: execute"bigger than memory" operations

http://dask.pydata.org/en/latest/

https://jakevdp.github.io/blog/2015/08/14/out-of-core-dataframes-in-python/

![](./images/dask_is_smart.jpg)

In [ ]:
import dask
from dask.async import get_sync
dask.set_options(get=get_sync);

The following lines do not download the data

In [ ]:
from netCDF4 import Dataset

nc = Dataset(url)

t = nc['temp']
s = nc['salt']

t.shape, s.shape

By making the netCDF4 variable a dask variable we can start computing stuff without loading the data!
Note

In [ ]:
import dask.array as da

chunks = (1, 16, 107, 345)

t = da.from_array(t, chunks=chunks)
s = da.from_array(s, chunks=chunks)

t

Note that the result is a dask element wise object

In [ ]:
(s + t) * 2

You need to manually trigger the computations to load the data.
Note that some operations, like plotting, will load the data automatically:

In [ ]:
import gsw
import seawater as sw
import numpy.ma as ma

S = s[0, -1, :, :]
T = t[0, -1, :, :]

sigma0 = gsw.sigma0(S.compute(), T.compute())
cs = plt.pcolormesh(ma.masked_equal(sigma0, -1000), cmap=plt.cm.viridis)

# cartopy: maps!

http://scitools.org.uk/cartopy/

The first python library to actually solve the dateline problem

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)
ax.coastlines(); ax.set_global()

kw = dict(lw=4, color='g', transform=ccrs.Geodetic())
l0 = plt.plot([-100, 50], [25, 25], label='GD1', **kw)
l1 = plt.plot([-38, 147], [-13, -42], label='GD2', **kw)

kw = dict(linewidth=4, color='b', transform=projection)
l2 = plt.plot([-100, 50], [25, 25], label='PC1', **kw)
l3 = plt.plot([-38, 147], [-13, -42], label='PC2', **kw)

leg = ax.legend(loc=(1.05, 0.5))

When creating a lot of maps it is convenient to wrap them in a plotting function

In [ ]:
from oceans import cm
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import (LONGITUDE_FORMATTER,
                                   LATITUDE_FORMATTER)

def make_map(projection=ccrs.PlateCarree()):
    subplot_kw = dict(projection=projection)
    fig, ax = plt.subplots(figsize=(9, 13),
                           subplot_kw=subplot_kw)
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
lon = nc['lon_rho'][:]
lat = nc['lat_rho'][:]

fig, ax = make_map()
ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()])
ax.coastlines('10m')
s0 = ma.masked_equal(sigma0, -1000)
cs = ax.pcolormesh(lon, lat, s0, cmap=cm.avhrr)
cbar = fig.colorbar(cs, shrink=0.45, extend='both')

# iris

http://scitools.org.uk/iris/docs/latest/index.html

Iris is an interpretation of the [CF-Conventions](http://cfconventions.org/).
The main object is the `cube`:

In [ ]:
import iris

cubes = iris.load_raw(url)
print(cubes)

We can take advantage of the CF conventions to access variables by its `standard_name`

In [ ]:
temp = cubes.extract_strict('sea_water_potential_temperature')
print(temp)

Like `xray` we have both high and low level slicing
(Although the high level slicing in `iris` is too sophisticated for my taste ;-)

In [ ]:
t = temp[-1, -1, ...]

print(t)  # The metadata is always propagated.

There are even some quick plotting routines to explore the data.
Note the free colorbar, units label, and title:

In [ ]:
import iris.quickplot as qplt

cs = qplt.pcolormesh(t, cmap=plt.cm.viridis)

# Optimizing code for iso-surfaces using Fortran, Cython, and Numba

In [ ]:
p = np.linspace(-100, 0, 30)[:, None, None] * np.ones((50, 70))
x, y = np.mgrid[0:20:50j, 0:20:70j]

q = np.sin(x) + p
p0 = -50.

In [ ]:
def naive_zslice(q, p, p0, mask_val=np.NaN):
    N, M, L = q.shape[0], q.shape[1], q.shape[2]
    
    q_iso = np.empty((M, L))
    for i in range(L):
        for j in range(M):
            q_iso[j, i] = mask_val
            for k in range(N-1):
                if (((p[k, j, i] < p0) and (p[k+1, j, i] > p0)) or
                    ((p[k, j, i] > p0) and (p[k+1, j, i] < p0))):
                    dp = p[k+1, j, i] - p[k, j, i]
                    dp0 = p0 - p[k, j, i]
                    dq = q[k+1, j, i] - q[k, j, i]
                    q_iso[j, i] = q[k, j, i] + dq*dp0/dp
    return q_iso

naive = %timeit -n1000 -o naive_zslice(q, p, p0)

In [ ]:
def numpy_zslice(q, p, p0):
    N, L, M = q.shape
    p0 = -abs(p0)
    data = q.reshape(N, -1, order='F')
    z = p.reshape(N, -1, order='F')

    bottom = np.zeros((1, L*M))
    top = np.empty_like(bottom)
    top.fill(-np.inf)
    z = np.r_[top, z, bottom]

    top.fill(np.NaN)
    data = np.r_[top, data, data[-1, ...][None, :]]
    z, data = map(np.flipud, (z, data))
    zg_ind = np.diff(z < p0, axis=0).ravel('F').nonzero()[0]
    zg_ind += np.arange(0, len(zg_ind), 1)
    depth_greater_z = z.ravel('F')[zg_ind]
    data_greater_z = data.ravel('F')[zg_ind]

    zl_ind = np.diff(z > p0, axis=0).ravel('F').nonzero()[0]
    zl_ind += np.arange(1, len(zg_ind)+1, 1)
    depth_lesser_z = z.ravel('F')[zl_ind]
    data_lesser_z = data.ravel('F')[zl_ind]

    alpha = (p0-depth_greater_z) / (depth_lesser_z-depth_greater_z)
    data_at_depth = (data_lesser_z*alpha) + (data_greater_z*(1-alpha))
    return data_at_depth.reshape(L, M, order='F')

In [ ]:
numpy = %timeit -n1000 -o numpy_zslice(q, p, p0)

## Let's bring the loop back!

![](./images/idiosyncratic.jpg)

In [ ]:
%load_ext fortranmagic

In [ ]:
%%fortran
      subroutine fortran_zslice(q, p, p0, q_iso, L, M, N)
      implicit none
      integer L, M, N
      real*8 q(N,M,L)
      real*8 p(N,M,L)
      real*8 q_iso(M,L)
cf2py intent(out) q_iso
      integer i, j, k
      real*8 dq, dp, dp0, p0

      do i=1,L
        do j=1,M
          q_iso(j,i)=1.0d20 ! default value - isoline not in profile
          do k=1,N-1
            if ( (p(k,j,i).lt.p0.and.p(k+1,j,i).gt.p0).or.
     &           (p(k,j,i).gt.p0.and.p(k+1,j,i).lt.p0) ) then
              dp = p(k+1,j,i) - p(k,j,i)
              dp0 = p0 - p(k,j,i)
              dq = q(k+1,j,i) - q(k,j,i)
              q_iso(j,i) = q(k,j,i) + dq*dp0/dp
            endif
          enddo
        enddo
      enddo

      return
      end subroutine fortran_zslice

In [ ]:
fotran = %timeit -n1000 -o fortran_zslice(q, p, p0)

In [ ]:
%load_ext Cython

In [ ]:
%%cython
cimport cython
import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def cython_zslice(double[:, :, ::1] q,
                    double[:, :, ::1] p,
                    double p0,
                    mask_val=np.NaN):
    cdef int L = q.shape[2]
    cdef int M = q.shape[1]
    cdef int N = q.shape[0]
    cdef double dp, dq, dq0
    cdef int i, j, k
    
    cdef double[:, ::1] q_iso = np.empty((M, L), dtype=np.float64)
    
    for i in range(L):
        for j in range(M):
            q_iso[j, i] = mask_val
            for k in range(N-1):
                if (((p[k, j, i] < p0) and (p[k+1, j, i] > p0)) or
                    ((p[k, j, i] > p0) and (p[k+1, j, i] < p0))):
                    dp = p[k+1, j, i] - p[k, j, i]
                    dp0 = p0 - p[k, j, i]
                    dq = q[k+1, j, i] - q[k, j, i]
                    q_iso[j, i] = q[k, j, i] + dq*dp0/dp
    return np.array(q_iso)

In [ ]:
cython = %timeit -n1000 -o cython_zslice(q, p, p0)

In [ ]:
from numba.decorators import jit

@jit
def numba_zslice(q, p, p0, mask_val=np.NaN):
    N, M, L = q.shape[0], q.shape[1], q.shape[2]
    
    q_iso = np.empty((M, L))
    for i in range(L):
        for j in range(M):
            q_iso[j, i] = mask_val
            for k in range(N-1):
                if (((p[k, j, i] < p0) and (p[k+1, j, i] > p0)) or
                    ((p[k, j, i] > p0) and (p[k+1, j, i] < p0))):
                    dp = p[k+1, j, i] - p[k, j, i]
                    dp0 = p0 - p[k, j, i]
                    dq = q[k+1, j, i] - q[k, j, i]
                    q_iso[j, i] = q[k, j, i] + dq*dp0/dp
    return q_iso

In [ ]:
numba = %timeit -n1000 -o numba_zslice(q, p, p0)

In [ ]:
from pandas import DataFrame

benchmarkings = dict(naive=naive.best,
                     numpy=numpy.best,
                     fortran=fotran.best,
                     cython=cython.best,
                     numba=numba.best)

benchmarkings = DataFrame.from_dict(benchmarkings, orient='index')
benchmarkings.sort_values(by=0, ascending=False, inplace=True)
ax = benchmarkings.plot(kind='bar', logy=True, legend=False)
yt = ax.set_ylabel('Times (ms)')

## How to decide which one use?

![](./images/good_code.png)

# Some very "oceanographic" libraries


- [odvc](https://raw.githubusercontent.com/ioos/odvc/master/notebooks/ocean_sigma_coordinate-FVCOM.ipynb)
- [pyoos](http://ioos.github.io/system-test/)
- [pyugrid](https://ocefpaf.github.io/python4oceanographers/blog/2015/07/20/pyugrid/)
- [pysgrid](http://nbviewer.ipython.org/urls/raw.githubusercontent.com/sgrid/pysgrid/master/pysgrid/notebook_examples/hudson_shelf_valley.ipynb)
- [ciso](http://nbviewer.ipython.org/gist/ocefpaf/ee14af5220bc3512131f)
- [utide](https://ocefpaf.github.io/python4oceanographers/blog/2015/05/18/utide_ellipse/)

# Using python as a glue language

## MatLab

![](./images/626.gif)

In [ ]:
try:
    %load_ext oct2py.ipython
    octave = True
except OSError as e:
    octave = False
    print(e)
    print("You cannot run the next 2 cells with the octave example :-(")

In [ ]:
%%file fahr_to_kelvin.m

function ktemp = fahr_to_kelvin(ftemp)
    ktemp = ((ftemp - 32) * (5/9)) + 273.15;
end

In [ ]:
if octave:
    %octave fahr_to_kelvin(32)

## Fortran

![](./images/Olde_Fortran_Malt_Liquor.jpg)

In [ ]:
%load_ext fortranmagic

In [ ]:
%%fortran

subroutine calc_sin(x, y)
    real, intent(in) :: x
    real, intent(out) :: y

    y = sin(x)

end subroutine calc_sin

In [ ]:
calc_sin(np.pi/2)

## R

![](./images/r_curve.jpg)

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
X=c(1,4,5,7)
Y = c(2,4,3,9)
print(summary(lm(Y~X)))

In [ ]:
%%R
library(oce)
Sa <- 30; Ta <- 10; Sb <- 40; 
rho0 <- swRho(Sa, Ta, 0)
Tb <- uniroot(function(T) rho0-swRho(Sb,T,0), lower=0, upper=100)$root
Sc <- (Sa + Sb) /2; Tc <- (Ta + Tb) /2
drho <- swRho(Sc, Tc, 0) - rho0
dT <- drho / rho0 / swAlpha(Sc, Tc, 0)
plotTS(as.ctd(c(Sa, Sb, Sc), c(Ta, Tb, Tc), 0), pch=20, cex=2)
drawIsopycnals(levels=rho0, col="red", cex=0)
segments(Sa, Ta, Sb, Tb, col="blue")
text(Sb, Tb, "b", pos=4); text(Sa, Ta, "a", pos=4); text(Sc, Tc, "c", pos=4)
legend("topleft",legend=sprintf("Sa=%.1f, Ta=%.1f, Sb=%.1f  ->  Tb=%.1f, drho=%.2f, dT=%.2f",Sa, Ta, Sb, Tb, drho, dT),bg="white")

## Caveat: Just because I said this is good stuff it means you should use it

![](./images/questionable_taste.jpg)

##  Now we are ready to fix the World!

In [ ]:
from skimage import io
from skimage.transform import rotate

img = io.imread('./images/dsc_0223.jpg')

rotated = rotate(img, -5, order=3)

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(9, 9))
ax0.imshow(img)
ax1.imshow(rotated);